In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt

from keras.models import load_model
from pandas.io.parsers import read_csv
from sklearn.utils import shuffle

### Load data

Utility

In [ ]:
def load_data(test=False):
    """
    Loads data from FTEST if *test* is True, otherwise from FTRAIN.
    The files are in a ../data/kaggle_facialKeyPointDetection/ directory
    """
    
    FTRAIN = '../data/kaggle_facialKeyPointDetection/training.csv'
    FTEST = '../data/kaggle_facialKeyPointDetection/test.csv'
    
    fname = FTEST if test else FTRAIN
    
    # Load dataframes
    df = read_csv(os.path.expanduser(fname))
    
    # Image column has space separated pixel values - converting values to numpy arrays
    df['Image'] = df['Image'].apply(lambda im: np.fromstring(im, sep=' '))

    # Drop rows that have missing values in them
    df = df.dropna()

    # Scale pixel values to [0, 1]
    X = np.vstack(df['Image'].values) / 255.  
    X = X.astype(np.float32)
    
    # return images as 96 x 96 x 1
    X = X.reshape(-1, 96, 96, 1)

    if not test:
        y = df[df.columns[:-1]].values
        y = (y - 48) / 48  # scale target coordinates to [-1, 1]
        X, y = shuffle(X, y, random_state=42)  # shuffle train data
        y = y.astype(np.float32)
    else:
        y = None

    return X, y

Load training data

In [ ]:
X_train, y_train = load_data()

## Facial landmark CNN model

CNN model architecture

In [ ]:
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Convolution2D, MaxPooling2D, Dropout
from keras.layers import Flatten, Dense
from keras.optimizers import SGD, RMSprop, Adagrad, Adadelta, Adam, Adamax, Nadam

In [ ]:
def get_my_CNN_model_architecture():
    '''
    The network accepts a 96x96 grayscale image as input and returns a vector with 30 entries
    corresponding to the predicted (horizontal and vertical) locations of 15 facial keypoints
    '''
    
    model = Sequential()
    model.add(Convolution2D(32, (5, 5), input_shape=(96,96,1), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

    model.add(Convolution2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.1))

    model.add(Convolution2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Convolution2D(30, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.3))

    model.add(Flatten())

    model.add(Dense(64, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(256, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(30))

    return model

Util - Calibrate model

In [ ]:
def compile_my_CNN_model(model, optimizer, loss, metrics):
    model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [ ]:
def train_my_CNN_model(model, X_train, y_train):
    return model.fit(X_train, y_train, epochs=100, batch_size=200, verbose=1, validation_split=0.2)

Build model

In [ ]:
my_model = get_my_CNN_model_architecture()

# Compiling the CNN model with an appropriate optimizer and loss and metrics
compile_my_CNN_model(my_model, optimizer = 'adam', loss = 'mean_squared_error', metrics = ['accuracy'])

# Training the model
hist = train_my_CNN_model(my_model, X_train, y_train)

Save model

In [ ]:
my_model.save('../model/my-facial_landmark-model.h5')

### Add filter to image

Load image

In [ ]:
my_image = cv2.imread('../../zzz/official-docs/myPhoto.png')

Convert image to gray scale

In [ ]:
gray = cv2.cvtColor(my_image, cv2.COLOR_BGR2GRAY)

Face cascade to detect faces

In [ ]:
face_cascade = cv2.CascadeClassifier('cascades/haarcascade_frontalface_default.xml')

Detect faces using Haar cascade object

In [ ]:
faces = face_cascade.detectMultiScale(gray, 1.25, 6)
faces

Load facial landmark model

In [ ]:
my_model = load_model('../model/my-facial_landmark-model.h5')

Load filters and select a filter

In [ ]:
filters = ['../resources/akshayChandra_selfie-filters/images/sunglasses_1.png', 
           '../resources/akshayChandra_selfie-filters/images/sunglasses_2.png', 
           '../resources/akshayChandra_selfie-filters/images/sunglasses_3.jpg', 
           '../resources/akshayChandra_selfie-filters/images/sunglasses_4.png', 
           '../resources/akshayChandra_selfie-filters/images/sunglasses_5.jpg', 
           '../resources/akshayChandra_selfie-filters/images/sunglasses_6.png']
filterIndex = 2

Get facial landmarks in ROI

In [ ]:
# Loop over all the faces found in the frame

for (x, y, w, h) in faces:
    
    # Make the faces ready for the model (normalize and resize)
    gray_face = gray[y:y+h, x:x+w]
    color_face = my_image[y:y+h, x:x+w]
    my_image_filter = my_image.copy()

    # Normalize to match the input format of the model - Range of pixel to [0, 1]
    gray_normalized = gray_face / 255

    # Resize it to 96x96 to match the input format of the model
    original_shape = gray_face.shape # A Copy for future reference
    face_resized = cv2.resize(gray_normalized, (96, 96), interpolation = cv2.INTER_AREA)
    face_resized_copy = face_resized.copy()
    face_resized = face_resized.reshape(1, 96, 96, 1)

    # Predict the keypoints using the model
    keypoints = my_model.predict(face_resized)

    # De-Normalize the keypoints values
    keypoints = keypoints * 48 + 48

    # Map the Keypoints back to the original image
    face_resized_color = cv2.resize(color_face, (96, 96), interpolation = cv2.INTER_AREA)
    face_resized_color2 = np.copy(face_resized_color)

    # Pair the keypoints together - (x1, y1)
    points = []
    for i, co in enumerate(keypoints[0][0::2]):
        points.append((co, keypoints[0][1::2][i]))
        
    # Add FILTER to the frame
    sunglasses = cv2.imread(filters[filterIndex], cv2.IMREAD_UNCHANGED)
    sunglass_width = int((points[7][0]-points[9][0])*1.1)
    sunglass_height = int((points[10][1]-points[8][1])/1.1)
    sunglass_resized = cv2.resize(sunglasses, 
                                  (sunglass_width, sunglass_height), interpolation = cv2.INTER_CUBIC)
    transparent_region = sunglass_resized[:,:,:3] != 0
    face_resized_color[int(points[9][1]):int(points[9][1])+sunglass_height, 
                       int(points[9][0]):
                       int(points[9][0]) + 
                       sunglass_width,:][transparent_region] = sunglass_resized[:,:,:3][transparent_region]

    # Resize the face_resized_color image back to its original shape
    my_image[y:y+h, x:x+w] = cv2.resize(face_resized_color, original_shape, interpolation = cv2.INTER_CUBIC)

    # Add KEYPOINTS to the my_image_filter
    for keypoint in points:
        cv2.circle(face_resized_color2, keypoint, 1, (0,255,0), 1)

    my_image_filter[y:y+h, x:x+w] = cv2.resize(face_resized_color2, original_shape, 
                                               interpolation = cv2.INTER_CUBIC)
    
    
    # If the 'q' key is pressed, stop the loop
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break

Inspect output

In [ ]:
# Show the frame and the my_image_filter
cv2.imshow("Selfie Filters", my_image)
cv2.imshow("Facial Keypoints", my_image_filter)
cv2.waitKey(0)